In [36]:
import numpy as np
import pandas as pd
import scipy.stats
import sklearn_crfsuite
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn_crfsuite import metrics

In [177]:
DATA = "PTP" 

if DATA == "PTP": 
    
    df_test = pd.read_csv('../../data_v3/ptp_il_tagged.csv')
    
    df_train = pd.read_csv('../../data_v3/ptp_extended_features.csv')
    
    df_test.rename(columns={'real_activity_action': "ActivityAction"}, inplace=True)
    
    df_gt = pd.read_csv('../../data_v3/ptp_ground_truth.csv')
    
elif DATA == "HR": 
    
    df_test = pd.read_csv('../../data_v3/hr_il_tagged.csv')

    df_train = pd.read_csv('../../data_v3/hr_extended_features.csv')
    df_test.drop(columns=['ActivityAction'], inplace=True)
    df_test.rename(columns={'real_activity_action': "ActivityAction"}, inplace=True)
    
    df_gt = pd.read_csv('../../data_v3/hr_ground_truth.csv')

In [178]:
# ------------ OPTIONAL ---------------
# TODO Duplicate with Activity Model - move down and delete
# Form sequences in training data by grouping
df_train = df_train.sort_values(by=["InstanceNumber", "BusinessActivity", "frame.number"])
df_train["SequenceNumber"] = df_train.groupby(["BusinessActivity", "InstanceNumber"]).ngroup()
df_train["SequenceNumber"] -= df_train['SequenceNumber'].min()

# check sequence length of training data
df_train["SequenceNumber"].value_counts()

SequenceNumber
256    198
313    198
273    198
311    198
266    198
      ... 
49       9
22       9
60       9
25       9
40       8
Name: count, Length: 409, dtype: int64

In [179]:
df_train[df_train["event_with_roles"] == "Odoo Application->db Server/Mail Server: [PgsqlRequest:Simple query:UPDATE:['fetchmail_server']]"][["frame.number",*cols,"ActivityAction", "SequenceNumber"]]


,frame.number,event_with_roles,request_method_call,selective_filter_data,origin_request_method,origin_selective_filter_data,ActivityAction,SequenceNumber
223,2209,Odoo Application->db Server/Mail Server: [Pgsq...,NaN,NaN,NaN,NaN,NoAction,0
62,1541,Odoo Application->db Server/Mail Server: [Pgsq...,NaN,NaN,NaN,NaN,NoAction,67
314,199,Odoo Application->db Server/Mail Server: [Pgsq...,NaN,NaN,NaN,NaN,NoAction,315
433,4075,Odoo Application->db Server/Mail Server: [Pgsq...,NaN,NaN,NaN,NaN,NoAction,362
495,1033,Odoo Application->db Server/Mail Server: [Pgsq...,NaN,NaN,NaN,NaN,NoAction,68
...,...,...,...,...,...,...,...,...
25747,408,Odoo Application->db Server/Mail Server: [Pgsq...,NaN,NaN,NaN,NaN,NoAction,66
25611,219,Odoo Application->db Server/Mail Server: [Pgsq...,NaN,NaN,NaN,NaN,NoAction,133
25836,6122,Odoo Application->db Server/Mail Server: [Pgsq...,NaN,NaN,NaN,NaN,Activity End,180
25729,5369,Odoo Application->db Server/Mail Server: [Pgsq...,NaN,NaN,NaN,NaN,NoAction,314


In [192]:
drop_val = "Odoo Application->db Server/Mail Server: [PgsqlRequest:Simple query:UPDATE:['fetchmail_server']]"

# Drop "Fetchmail server from Training Data 
df_train_filt = df_train[df_train["event_with_roles"] != drop_val]

In [193]:
cols = ["event_with_roles", "request_method_call", "selective_filter_data","origin_request_method","origin_selective_filter_data"]

cols_ext = ["frame.number",*cols,"ActivityAction", "SequenceNumber"]

In [194]:
def mark_start_end(df):
    # Mark start event of each BusinessActivity Instance
    df["activityStart"] = df.groupby(["BusinessActivity", "InstanceNumber", ]).cumcount() == 0
    # Mark end event of each Business Activity Instance
    df["activityEnd"] = df.groupby(["BusinessActivity", "InstanceNumber", ]).cumcount(ascending=False) == 0
    # Merge start and end columns to form labels
    df["ActivityAction"] = df.apply(lambda row: "Activity Start" if row["activityStart"] else (
        "Activity End" if row["activityEnd"] else 'NoAction'), axis=1)

    return df.drop(["activityStart", 'activityEnd'], axis=1)

df_train = mark_start_end(df_train_filt)

/var/folders/qt/gt7q1tn978x_02pv47b2lrvm0000gn/T/ipykernel_9924/779422405.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["activityStart"] = df.groupby(["BusinessActivity", "InstanceNumber", ]).cumcount() == 0
/var/folders/qt/gt7q1tn978x_02pv47b2lrvm0000gn/T/ipykernel_9924/779422405.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["activityEnd"] = df.groupby(["BusinessActivity", "InstanceNumber", ]).cumcount(ascending=False) == 0
/var/folders/qt/gt7q1tn978x_02pv47b2lrvm0000gn/T/ipykernel_9924/7

In [195]:
def dict_to_features(dict):
    return [[{**d, "bias": 1.0}] for d in dict]


def extract_labels(labels):
    return [[y] for y in labels]

In [196]:
#cols = ["event_with_roles", "request_method_call", "selective_filter_data","origin_selective_filter_data"]
train_features = df_train[cols].to_dict("records")
train_features = dict_to_features(train_features)
train_labels = extract_labels(df_train["ActivityAction"])

In [197]:
test_features = df_test[cols].to_dict("records")
test_features = dict_to_features(test_features)
test_labels = extract_labels(df_test["ActivityAction"])

In [198]:
%%time
import sklearn_crfsuite

crf = sklearn_crfsuite.CRF(
    max_iterations=200,
    c1=0.1,
    c2=0.01,
    all_possible_transitions=True
    #all_possible_transitions=True
)
crf.fit(train_features, train_labels)

CPU times: user 796 ms, sys: 9.94 ms, total: 806 ms
Wall time: 1.27 s


CRF(all_possible_transitions=True, c1=0.1, c2=0.01, max_iterations=200)

In [199]:
from sklearn.metrics import multilabel_confusion_matrix
from sklearn_crfsuite import metrics


def flatten(xss):
    return [x for xs in xss for x in xs]


def evaluate(model, x, y_true):
    y_pred = model.predict(x)
    print(metrics.flat_f1_score(y_true, y_pred, average='macro', labels=model.classes_))
    print(metrics.flat_classification_report(y_true, y_pred, model.classes_))
    [print(label, "\n", matrix) for matrix, label in
     zip(multilabel_confusion_matrix(flatten(y_true), flatten(y_pred), labels=model.classes_), model.classes_)]


evaluate(crf, test_features, test_labels)

0.8487744079767344
                precision    recall  f1-score   support

Activity Start       1.00      1.00      1.00        63
      NoAction       0.99      1.00      0.99      3591
  Activity End       1.00      0.38      0.55        63

      accuracy                           0.99      3717
     macro avg       1.00      0.79      0.85      3717
  weighted avg       0.99      0.99      0.99      3717

Activity Start 
 [[3654    0]
 [   0   63]]
NoAction 
 [[  87   39]
 [   0 3591]]
Activity End 
 [[3654    0]
 [  39   24]]


In [129]:
from sklearn.metrics import make_scorer
import scipy
from sklearn.model_selection import RandomizedSearchCV
from sklearn_crfsuite import metrics

# define fixed parameters and parameters to search
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=200,
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='macro', labels=np.unique(test_labels))

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=5,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=150,
                        scoring=f1_scorer)
rs.fit(train_features, train_labels)

crf = rs.best_estimator_

Fitting 5 folds for each of 150 candidates, totalling 750 fits


In [131]:
from collections import Counter

def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))


print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))

print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])

Top positive:
9.441744 Activity End origin_selective_filter_data:sale.order.line_create
8.230058 Activity End origin_selective_filter_data:account.payment_create
7.622693 Activity Start request_method_call:version
7.622693 Activity Start selective_filter_data:version
7.435797 NoAction bias
6.027338 Activity End request_method_call:faultCode
6.027338 Activity End selective_filter_data:faultCode
4.309039 Activity End event_with_roles:Odoo Application->db Server/Mail Server: [PgsqlRequest:Simple query:UPDATE:['mail_mail']]
4.091216 Activity End origin_selective_filter_data:purchase.requisition_write_ongoing
3.416046 Activity End selective_filter_data:IsNumber
2.960928 Activity End origin_selective_filter_data:purchase.requisition_write_open
2.592821 NoAction request_method_call:2
1.963039 Activity End origin_selective_filter_data:purchase.order_write
1.146282 NoAction origin_request_method:execute_kw
0.890859 Activity End request_method_call:1
0.758337 Activity End event_with_roles:Odoo A